In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy
import os
import cv2
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

SETUP PATH


In [2]:
CUSTOM_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite'

In [3]:
WORKSPACE_PATH = 'C:/Users/ADMIN/Documents/TensorFlow/workspace/training_demo'
SCRIPTS_PATH = 'C:/Users/ADMIN/Documents/TensorFlow/script'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/'+ CUSTOM_MODEL_NAME +'/'

In [4]:
CONFIG_PATH = MODEL_PATH +'/'+ CUSTOM_MODEL_NAME +'/pipeline.config'

Label Map

In [5]:
#Tạo label map
category_index = {
    1: {'id': 1, 'name': 'Ferrari'},
    2: {'id': 2, 'name': 'Ford'},
    3: {'id': 3, 'name': 'Nbc'},
    4: {'id': 4, 'name': 'Starbucks'},
    5: {'id': 5, 'name': 'RedBull'},
    6: {'id': 6, 'name': 'Mini'},
    7: {'id': 7, 'name': 'Unicef'},
    8: {'id': 8, 'name': 'Yahoo'},
    9: {'id': 9, 'name': 'Sprite'},
    10: {'id': 10, 'name': 'Texaco'},
    11: {'id': 11, 'name': 'Intel'},
    12: {'id': 12, 'name': 'Cocacola'},
    13: {'id': 13, 'name': 'Citroen'},
    14: {'id': 14, 'name': 'Heineken'},
    15: {'id': 15, 'name': 'Apple'},
    16: {'id': 16, 'name': 'Google'},
    17: {'id': 17, 'name': 'Fedex'},
    18: {'id': 18, 'name': 'Pepsi'},
    19: {'id': 19, 'name': 'Puma'},
    20: {'id': 20, 'name': 'DHL'},
    21: {'id': 21, 'name': 'Porsche'},
    22: {'id': 22, 'name': 'Nike'},
    23: {'id': 23, 'name': 'Vodafone'},
    24: {'id': 24, 'name': 'BMW'},
    25: {'id': 25, 'name': 'McDonalds'},
    26: {'id': 26, 'name': 'HP'},
    27: {'id': 27, 'name': 'Adidas'},
}

CONFIG


In [6]:
#Dọc pipeline config của model
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 27
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [7]:
#Load pipeline config để tùy chỉnh model
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [8]:
#Tùy chỉnh các param của model
pipeline_config.model.ssd.num_classes = 27 #Số lượng brand
pipeline_config.train_config.batch_size = 16 
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite/checkpoint/ckpt-0' #Load checkpoint cho transfer learning
pipeline_config.train_config.fine_tune_checkpoint_type = "detection" 
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt' #Đường dẫn đến file label cho input
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH +'/train.tfrecord'] #Đường dẫn đến file input
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt' #Đường dẫn đến file label cho output
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH +"/test" + '/test*.tfrecord'] #Đường dẫn đến file test

In [9]:
#Viết lại thông tin vào file config
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

TRAIN

In [10]:
print("python model_main_tf2.py --model_dir=models/"+CUSTOM_MODEL_NAME+" --pipeline_config_path=models/"+CUSTOM_MODEL_NAME+"/pipeline.config")

python model_main_tf2.py --model_dir=models/ssd_mobilenet_v2_fpnlite --pipeline_config_path=models/ssd_mobilenet_v2_fpnlite/pipeline.config


Monitor

In [11]:
print("tensorboard --logdir=models/"+CUSTOM_MODEL_NAME)

tensorboard --logdir=models/ssd_mobilenet_v2_fpnlite


Evaluate

In [12]:
print("python model_main_tf2.py --model_dir=models/"+CUSTOM_MODEL_NAME+" --pipeline_config_path=models/"+CUSTOM_MODEL_NAME+"/pipeline.config --checkpoint_dir=models/"+CUSTOM_MODEL_NAME)

python model_main_tf2.py --model_dir=models/ssd_mobilenet_v2_fpnlite --pipeline_config_path=models/ssd_mobilenet_v2_fpnlite/pipeline.config --checkpoint_dir=models/ssd_mobilenet_v2_fpnlite


In [13]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-51')).expect_partial()

In [14]:
"""
    Detection
    
    Parameter
    ----------------
    - image

    Returns
    ----------------
    - detection  
        + Bounding boxes(xmin,ymin,xmax,ymax): hình chữ nhật chứa object
        + Score: Độ tin cậy của bounding box
        + Class: Class của object dã detect được 
"""
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [21]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   
IMAGE_PATHS = ["C:/Users/ADMIN/Documents/TensorFlow/workspace/training_demo/images/cocacola.jpg"]
def load_image_into_numpy_array(path):
    """
    Load ảnh

    Param:
    ----------
        path: Đường dẫn đến image

    Returns:
    ---------
        uint8 numpy array với kích thước (height, width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    image_np = load_image_into_numpy_array(image_path)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32) #Chuyển ảnh về tensor
    
    detections = detect_fn(input_tensor)

    # Lấy thông tin từ detection
    # Output là một batch tensor, phần tử đầu tiên của batch tensor 
    num_detections = int(detections.pop('num_detections')) #Lấy số lượng object detect được
    detections = {key: value[0, :num_detections].numpy() #Chuyển đổi tensor trong detection thành array
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections #Cập nhập số lượng detection
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    #Vẽ bounding box lên ảnh
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.5,
            agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()
cv2.imshow("img",image_np_with_detections)
cv2.waitKey()
cv2.destroyAllWindows()
# sphinx_gallery_thumbnail_number = 2

Done


In [20]:
import cv2 
import numpy as np
# Setup capture
#"C:/Users/ADMIN/Documents/TensorFlow/workspace/training_demo/video/mcdonald.mp4"
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
        
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=10,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
detections = detect_fn(input_tensor)

KeyboardInterrupt: 